In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [5]:
class CnnModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.vocab_size = 6002
        self.embed_size = 100
        self.num_filters = 100
        self.filter_sizes = [3, 4, 5]
        self.output_classes = 2
        self.dropout = 0.8

        # Embedding layer
        self.embedding = nn.Embedding(self.vocab_size, self.embed_size)
        self.convs = nn.ModuleList([
                                    nn.Conv2d(
                                        in_channels=1, 
                                        out_channels=self.num_filters,
                                        kernel_size=(fs, self.embed_size)) 
                                    for fs in self.filter_sizes
        ])
        self.fc = nn.Linear(len(self.filter_sizes) * self.num_filters, self.output_classes)
        self.dropout = nn.Dropout(self.dropout)

    def forward(self, text, text_lengths):
        embedded = self.embedding(text)
        embedded = embedded.unsqueeze(1)

        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]

        cat = self.dropout(torch.cat(pooled, dim=1))
        return self.fc(cat)

In [6]:
model = CnnModel()

In [7]:
model.load_state_dict(torch.load('/content/drive/MyDrive/Cryptofuture/cnn_sentiment_1.pt', map_location='cpu'))

<All keys matched successfully>

In [8]:
model.parameters()

<generator object Module.parameters at 0x7f92f2cc0dd0>

In [1]:
import pickle

fh = open('/content/drive/MyDrive/Cryptofuture/text_vocab.pickle', 'rb');
text = pickle.load(fh)

In [3]:
text['hello'], len(text)

(791, 6002)

In [10]:
text['<pad>']

1

In [9]:
import spacy

nlp = spacy.load('en_core_web_sm')

In [22]:
def predict_sentiment(model, tweet, min_length=10):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(tweet)]
    if len(tokenized) < min_length:
        tokenized += ['<pad>'] * (min_length - len(tokenized))
    indexed = [text[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to('cpu')
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor, len(tensor)))
    return prediction

In [28]:
predict_sentiment(model, 'Retweet if you\'re putting your stimulus check in bitcoin')

tensor([[0.1630, 0.7539]], grad_fn=<SigmoidBackward>)

In [26]:
predict_sentiment(model, 'Russell Okung announced in December he was converting half of his $13M salary into Bitcoin.')

tensor([[0.4738, 0.5171]], grad_fn=<SigmoidBackward>)